<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/contrast/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile

zip_path = "/content/drive/MyDrive/AIEngineer/contrast/pipeline/TMTRF20250303.zip"

# Open the ZIP file and list its contents
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

print("Files in ZIP:")
for file in file_list:
    print(file)

Files in ZIP:
TMT in summary V3.pdf
TMTRF20250303/
TMTRF20250303/TMTRF20250303_DELTA.xls
TMTRF20250303/TMTRF20250303_FULL.xls
TMTRF20250303/TMTRF20250303_SNAPSHOT.xls
TMTRF20250303/TMTRF20250303_SNAPSHOT_GP_F.xls
TMTRF20250303_BONUS/
TMTRF20250303_BONUS/Concept/
TMTRF20250303_BONUS/Concept/GP20250303.xls
TMTRF20250303_BONUS/Concept/GPP20250303.xls
TMTRF20250303_BONUS/Concept/GPU20250303.xls
TMTRF20250303_BONUS/Concept/SUBS20250303.xls
TMTRF20250303_BONUS/Concept/TP20250303.xls
TMTRF20250303_BONUS/Concept/TPP20250303.xls
TMTRF20250303_BONUS/Concept/TPU20250303.xls
TMTRF20250303_BONUS/Concept/VTM20250303.xls
TMTRF20250303_BONUS/MasterTMT_20250303.xls
TMTRF20250303_BONUS/Relationship/
TMTRF20250303_BONUS/Relationship/GPPtoGPP20250303.xls
TMTRF20250303_BONUS/Relationship/GPPtoTPP20250303.xls
TMTRF20250303_BONUS/Relationship/GPtoGPU20250303.xls
TMTRF20250303_BONUS/Relationship/GPtoTP20250303.xls
TMTRF20250303_BONUS/Relationship/GPUtoGPP20250303.xls
TMTRF20250303_BONUS/Relationship/GPUtoTPU2

In [3]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
import zipfile
import pandas as pd
from neo4j import GraphDatabase

zip_path = "/content/drive/MyDrive/AIEngineer/contrast/pipeline/TMTRF20250303.zip"
extract_path = "/content/drive/MyDrive/AIEngineer/contrast/tmt/"

def extract_latest_files(zip_path, extract_path, date_suffix="0303"):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if date_suffix in file:
                zip_ref.extract(file, extract_path)

def load_excel_files(base_path, date_suffix):
    files = {
        "GP": f"{base_path}/Concept/GP{date_suffix}.xls",
        "VTM": f"{base_path}/Concept/VTM{date_suffix}.xls",
        "SUBS": f"{base_path}/Concept/SUBS{date_suffix}.xls",
        "TP": f"{base_path}/Concept/TP{date_suffix}.xls",
        "TPU": f"{base_path}/Concept/TPU{date_suffix}.xls",
        "GPU": f"{base_path}/Concept/GPU{date_suffix}.xls",
        "tp_tpu": f"{base_path}/Relationship/TPtoTPU{date_suffix}.xls",
        "gp_tp": f"{base_path}/Relationship/GPtoTP{date_suffix}.xls",
        "vtm_gp": f"{base_path}/Relationship/VTMtoGP{date_suffix}.xls",
        "subs_vtm": f"{base_path}/Relationship/SUBStoVTM{date_suffix}.xls",
        "gp_gpu": f"{base_path}/Relationship/GPtoGPU{date_suffix}.xls"
    }
    return {key: pd.read_excel(path) for key, path in files.items()}

def process_data(data):
    merge_df = data['tp_tpu'].merge(data['gp_tp'], on='TPID', how='left')
    merge_df = merge_df.merge(data['vtm_gp'], on='GPID', how='left')
    merge_df = merge_df.merge(data['subs_vtm'], on='VTMID', how='left')
    merge_df = merge_df.merge(data['gp_gpu'][['GPID', 'GPUID']], on='GPID', how='left')
    merge_df = merge_df.rename(columns={
        'GPID': 'TMTID(GP)', 'TPID': 'TMTID(TP)', 'GPUID': 'TMTID(GPU)', 'TPUID': 'TMTID(TPU)',
        'VTMID': 'TMTID(VTM)', 'SUBSID': 'TMTID(SUBS)'
    })
    for key in ['GP', 'TP', 'GPU', 'TPU', 'VTM', 'SUBS']:
        merge_df = merge_df.merge(data[key][['TMTID(' + key + ')', 'FSN']], on='TMTID(' + key + ')', how='left')
        merge_df = merge_df.rename(columns={'FSN': key + 'NAME'})
    merge_df = merge_df.groupby(['TMTID(TPU)', 'TPUNAME','TMTID(TP)', 'TPNAME', 'TMTID(GPU)', 'GPUNAME',
                                   'TMTID(GP)', 'GPNAME','TMTID(VTM)','VTMNAME'], as_index=False).agg({
        'TMTID(SUBS)': list, 'SUBSNAME': list
    })
    merge_df = merge_df.rename(columns={'TMTID(SUBS)': 'SUBSID_LIST', 'SUBSNAME': 'SUBS_LIST'})
    return merge_df

def update_neo4j(df):
    uri = "neo4j+s://707666c5.databases.neo4j.io"
    user = "neo4j"
    password = "A_v96H_jhZ7rLXGYAqJa-hN8_fIVlGbjt4IQM3cJE18"
    driver = GraphDatabase.driver(uri, auth=(user, password))

    with driver.session() as session:
        session.run("""
            CREATE CONSTRAINT IF NOT EXISTS ON (n:DRUG) ASSERT n.`TMTID(GP)` IS UNIQUE;
            CREATE CONSTRAINT IF NOT EXISTS ON (n:SUBS) ASSERT n.`TMTID(SUBS)` IS UNIQUE;
            CREATE CONSTRAINT IF NOT EXISTS ON (n:VTM) ASSERT n.`TMTID(VTM)` IS UNIQUE;
            CREATE CONSTRAINT IF NOT EXISTS ON (n:TP) ASSERT n.`TMTID(TP)` IS UNIQUE;
            CREATE CONSTRAINT IF NOT EXISTS ON (n:GP) ASSERT n.`TMTID(GP)` IS UNIQUE;
        """)
        for _, row in df.iterrows():
            session.run("""
                MERGE (n:DRUG { `TMTID(GP)`: $gp_id })
                SET n += { `TMTID(TPU)`: $tpu_id, `TPUNAME`: $tpu_name, `TMTID(TP)`: $tp_id, `TPNAME`: $tp_name,
                           `TMTID(GPU)`: $gpu_id, `GPUNAME`: $gpu_name, `GPNAME`: $gp_name,
                           `TMTID(VTM)`: $vtm_id, `VTMNAME`: $vtm_name, `SUBSID_LIST`: $subs_list, `SUBS_LIST`: $subs_names }

                MERGE (s:SUBS { `TMTID(SUBS)`: $subs_list })
                SET s.`SUBSNAME` = $subs_names

                MERGE (v:VTM { `TMTID(VTM)`: $vtm_id })
                SET v.`VTMNAME` = $vtm_name

                MERGE (g:GP { `TMTID(GP)`: $gp_id })
                SET g.`GPNAME` = $gp_name

                MERGE (t:TP { `TMTID(TP)`: $tp_id })
                SET t.`TPNAME` = $tp_name

                MERGE (n)-[:HAS_SUBSTANCE]->(s)
                MERGE (n)-[:CONTAINS_VTM]->(v)
                MERGE (n)-[:HAS_GENERIC_NAME]->(g)
                MERGE (g)-[:HAS_TRADE_NAME]->(t)
            """,
            gp_id=row['TMTID(GP)'], tpu_id=row['TMTID(TPU)'], tpu_name=row['TPUNAME'],
            tp_id=row['TMTID(TP)'], tp_name=row['TPNAME'], gpu_id=row['TMTID(GPU)'],
            gpu_name=row['GPUNAME'], gp_name=row['GPNAME'], vtm_id=row['TMTID(VTM)'],
            vtm_name=row['VTMNAME'], subs_list=row['SUBSID_LIST'], subs_names=row['SUBS_LIST']
        )
    driver.close()

# Run Pipeline
date_suffix = "20250303"
extract_latest_files(zip_path, extract_path, date_suffix)
data = load_excel_files(extract_path, date_suffix)
merge_df = process_data(data)
update_neo4j(merge_df)
